# Calculate odds ratios
Sample code to calculate odds ratios, p-values, and 95% for rates
of chemosensory changes in COVID patients during successive waves of COVID.

Similar to the analysis published in [Decreasing Incidence of Chemosensory Changes by COVID-19 Variant](https://journals.sagepub.com/doi/abs/10.1177/01945998221097656),
but using different intervals.


In [ ]:
from math import isclose
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
import warnings; warnings.simplefilter('ignore')

In [ ]:
def calculate_odds_ratio(two_by_two: pd.DataFrame):
    """Calculates the odds ratio, 95% CI, and p-values for a 2x2 matrix"""

    fisher_or, pvalue = fisher_exact(two_by_two)
    res = odds_ratio(two_by_two)
    contingency_or = res.statistic
    lower_ci, upper_ci = res.confidence_interval()

    # Round values to four significant digits.
    fisher_or, pvalue, lower_ci, upper_ci = tuple(
        [round(x,4) for x in (fisher_or, pvalue, lower_ci, upper_ci)])
    return fisher_or, pvalue, (lower_ci, upper_ci)

def get_or_by_wave(df):
    """Calculate odds ratios for successive waves compared to initial wave."""
    waves = list(df.wave)
    base_wave = waves[0]
    wave_stats = []
    cols = ['wave', 'non_st_loss', 'st_loss']
    for wave in waves:
        two_by_two = (df[df.wave.isin([base_wave, wave])][cols]
                .set_index('wave').T)

        # Calculate OR
        if wave != base_wave:
            odds, p, ci = calculate_odds_ratio(two_by_two)
        else:
            odds = 1
            p = ci = '-'

        st_loss = two_by_two[wave].min()
        non_st_loss = two_by_two[wave].max()
        total = two_by_two[wave].sum()
        prevalence = (st_loss / total).round(4)
        wave_stats.append(
            [st_loss, non_st_loss, total, prevalence, odds, p, str(ci)])

    cols = ['st_loss', 'non_st_loss', 'total', 'prevalence', 'odds_ratio', 
            'p_value', 'CI_95']
    wave_stats = pd.DataFrame(wave_stats, columns=cols, index=waves)
    wave_stats = wave_stats.reset_index().rename(columns={'index':'wave'})
    return wave_stats

def sig(p):
    "Gets * string corresponding to the significance of the p-value"
    p = float(p)
    if p < 0.001:
        return '***'
    elif p < 0.01:
        return '**'
    elif p < 0.05:
        return '*'
    return ''

In [ ]:
# Calculate odds ratios in COVID+ cohort with alpha as baseline.
df = pd.read_csv("data/st_wave_covid.csv").sort_values('ordinal')
df = df[df.ordinal > 0] # Remove "untyped" wave
covid_alpha_or = get_or_by_wave(df)
# covid_alpha_or.to_csv('out/covid_alpha_or.csv', index=False)
covid_alpha_or

In [ ]:
# COVID- alpha baseline
df = pd.read_csv("data/st_wave_control.csv").sort_values('ordinal')
df = df[df.ordinal > 0] # Remove "untyped" wave
control_alpha_or = get_or_by_wave(df)
# control_alpha_or.to_csv('out/control_alpha_or.csv', index=False)
control_alpha_or

In [ ]:
# Progressively use each wave as a baseline for calculating odds ratios
all_waves = pd.read_csv("data/st_wave_covid.csv").sort_values('ordinal')
final = all_waves[['wave', 'prevalence']]

for i, wave in enumerate(all_waves.wave):
    if wave == list(all_waves.wave)[-1]:
        break
    wave_subset = all_waves[all_waves.ordinal >= i]
    df = get_or_by_wave(wave_subset)
    all_odds = df[['odds_ratio', 'p_value']].values[1:]
    ors = [(str(odds) + sig(p)) for odds, p in all_odds]
    col = wave.replace('wave', 'base')
    final[col] = (['-'] * (i+1)) + ors
final